In [0]:
#import libraries
from pyspark.sql.functions import col
from pyspark.sql import functions as F

from pyspark.sql.functions import substring
import pandas as pd
import os
import glob2
import datetime
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,when,lit
import xlrd

In [0]:
#Get Brand FILE NAME File
BRAND_FILE_DIR_PATH = "/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/" + str(datetime.datetime.today().year)
BRAND_FILE_LIST = glob2.glob(BRAND_FILE_DIR_PATH + "/*Brand*")
print(BRAND_FILE_LIST)
BRAND_FILE_DIR_IDX = [int((i.split("1-"))[1].split("Blob")[0])  for i in BRAND_FILE_LIST]
# print(BRAND_FILE_DIR_IDX)
CURR_PERIOD_BRAND_FILE_NAME = BRAND_FILE_LIST[BRAND_FILE_DIR_IDX.index(max(BRAND_FILE_DIR_IDX))]
print("CURR_PERIOD_BRAND_FILE_NAME ",CURR_PERIOD_BRAND_FILE_NAME)

# To open Workbook 
wb = xlrd.open_workbook(CURR_PERIOD_BRAND_FILE_NAME) 

['/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2021/Brand Data P1-13Blob Upload YTD.xlsx']
CURR_PERIOD_BRAND_FILE_NAME /dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2021/Brand Data P1-13Blob Upload YTD.xlsx

In [0]:
SHEET_NAMES = wb.sheet_names()
print("SHEET_NAMES",SHEET_NAMES)
PERIOD_NUMBERS = []
YEAR_NUMBERS = []
for item in SHEET_NAMES:
    for subitem in item.split(' '):
#       print(subitem)
      if "P" in subitem:
        PERIOD_NUMBERS.append(int(subitem.split('P')[1]))
      else:
        YEAR_NUMBERS.append(int(subitem))
print("PERIOD_NUMBERS",PERIOD_NUMBERS)
print("YEAR_NUMBERS",YEAR_NUMBERS)

CURR_PERIOD = PERIOD_NUMBERS[-1]
CURR_PERIOD = (PERIOD_NUMBERS[-1])

PREV_PERIOD = CURR_PERIOD - 1


print("CURR_PERIOD" ,CURR_PERIOD)
print("PREV_PERIOD",PREV_PERIOD)

#Extracting Year
Year = YEAR_NUMBERS[-1]

print(Year)

SHEET_NAMES ['P1 2019', 'P2 2019', 'P3 2019', 'P4 2019', 'P5 2019', 'P6 2019', 'P7 2019', 'P8 2019', 'P9 2019', 'P10 2019', 'P11 2019', 'P12 2019', 'P13 2019', 'P1 2020', 'P2 2020', 'P3 2020', 'P4 2020', 'P5 2020', 'P6 2020', 'P7 2020', 'P8 2020', 'P9 2020', 'P10 2020', 'P11 2020', 'P12 2020', 'P13 2020']
PERIOD_NUMBERS [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
YEAR_NUMBERS [2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020]
CURR_PERIOD 13
PREV_PERIOD 12
2020

In [0]:
#Read into multiple data frames and append them together
ACTDF = pd.DataFrame()

for sheet_name in SHEET_NAMES:
  sheet_variable = sheet_name.replace(' ','_')
  print("sheet_variable",sheet_variable)
  globals()[sheet_variable] = pd.read_excel(CURR_PERIOD_BRAND_FILE_NAME,sheet_name,parse_cols = "A:K",skiprows = 8,names=['Brand_Seg','Seg','Brand','Market','KPI','Periodic','Period_Plan','YTD','YTD_Plan','MAT','MAT_Plan'])
  sheet_name_period = sheet_name.split(' ')[0]
#   print(sheet_name_period)
  PER_NUMBER =int(sheet_name_period.split('P')[1])
  
  sheet_name_year = sheet_name.split(' ')[1]
#   print(sheet_name_year)
  YEAR_NUMBER = int(sheet_name_year)
  
  
#   print("PER_NUMBER",PER_NUMBER)
#   print("YEAR_NUMBER",YEAR_NUMBER)
  globals()[sheet_variable]['PERIOD'] = PER_NUMBER
  globals()[sheet_variable]['YEAR'] = YEAR_NUMBER                                                                                                                         
# 
  ACTDF = ACTDF.append(globals()[sheet_variable])

sheet_variable P1_2019
sheet_variable P2_2019
sheet_variable P3_2019
sheet_variable P4_2019
sheet_variable P5_2019
sheet_variable P6_2019
sheet_variable P7_2019
sheet_variable P8_2019
sheet_variable P9_2019
sheet_variable P10_2019
sheet_variable P11_2019
sheet_variable P12_2019
sheet_variable P13_2019
sheet_variable P1_2020
sheet_variable P2_2020
sheet_variable P3_2020
sheet_variable P4_2020
sheet_variable P5_2020
sheet_variable P6_2020
sheet_variable P7_2020
sheet_variable P8_2020
sheet_variable P9_2020
sheet_variable P10_2020
sheet_variable P11_2020
sheet_variable P12_2020
sheet_variable P13_2020

In [0]:
#Remove unnecessary columns
ACTDF = ACTDF.replace('#Missing',0)
ACTDF = ACTDF.replace('-',0)
ACTDF.fillna(0, inplace=True)
ACTDF.drop(['Brand_Seg'], axis=1, inplace=True)
ACTDF.columns = ['Seg', 'Brand', 'Market', 'KPI', 'Period_ACT','Period_REPLAN','YTD_ACT','YTD_REPLAN','MAT_ACT','MAT_REPLAN','PERIOD','YEAR' ]
ACTDF.dropna(subset=['Seg', 'Brand', 'Market'], inplace=True)
ACTDF.head(1)

ACTDF = ACTDF.drop_duplicates()

In [0]:
ACTDF.loc[ACTDF["KPI"]=="MS.Net Sales 3rd Party","KPI"]="MS.Net Sales 3rd Pty"
ACTDF.loc[ACTDF["KPI"]=="MS.Trade Expenditures","KPI"]="MS.Trade Expenditure"
ACTDF.loc[ACTDF["KPI"]=="MS.Overhead Costs","KPI"]="MS.Overheads Costs"
ACTDF.loc[ACTDF["KPI"]=="MS.Other Franch & Sales Cost","KPI"]="MS.Other Franch&Sales Cost"

## Escalation Testing

In [0]:
Log = spark.read.format("csv").option("header", "true").option("inferschema","true").load("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/Escalation_Log.csv")

Log = Log.withColumn("Count",col("Count").cast(IntegerType()))

In [0]:
# display(Log)

In [0]:
##### To adjust the counter if needed.

# Log = Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),0).otherwise(col("Count")))

# Log.coalesce(1).write.mode('overwrite').csv("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv",header=True)
# CSV_DIR = "dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"
# FILES = [str(x.name) for x in dbutils.fs.ls("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv")]
# for i in range(len(FILES)):
#   if(FILES[i].startswith('part')):
#     dbutils.fs.cp(CSV_DIR  + 'Escalation_Log_1.csv/' + FILES[i],CSV_DIR + FILES[i])
#     dbutils.fs.rm(CSV_DIR+'Escalation_Log_1.csv' , True)
#     dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'Escalation_Log_1.csv')
    
# dbutils.fs.mv(CSV_DIR+'Escalation_Log_1.csv',CSV_DIR+'Escalation_Log.csv',recurse=True)

In [0]:
marsCAL = spark.read.format("csv").option("header", "true").option("delimiter",",").load("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/GCAL.csv")

datetoday=datetime.datetime.now()
datestring=datetoday.strftime("%m/%d/%Y")
month=datestring.split('/')[0]
day=datestring.split('/')[1]
year=datestring.split('/')[2]
month+day+year

df=marsCAL.filter(col('GCALDAY')==year+month+day).select('GMARSYEAR','GMARSPINY','GMARSWINP')

cur_year=int(df.agg({"GMARSYEAR":"max"}).collect()[0][0])
cur_period=int(df.filter(col("GMARSYEAR")==cur_year).agg({"GMARSPINY":"max"}).collect()[0][0])
cur_week=int(df.filter((col("GMARSYEAR")==cur_year)&(col("GMARSPINY")==cur_period)).agg({"GMARSWINP":"max"}).collect()[0][0])

ref_period=cur_period-1

cur_year,cur_period,cur_week,ref_period


Out[ 10 ]: (2021, 3, 1, 2)

In [0]:
max_year= ACTDF['YEAR'].max()
max_period= ACTDF.loc[ACTDF['YEAR']==max_year,'PERIOD'].max()

print("Max Period Available",max_period)

if cur_week==1:
  if (max_period==ref_period-1 ) & (max_year==cur_year):
    Log=Log.withColumn("Flag",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),lit("1")).otherwise(col("Flag")))\
         .withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),0).otherwise(col("Count")))
    print("QC Check pass for Brand Data File")
    
    Log.coalesce(1).write.mode('overwrite').csv("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv",header=True)
    CSV_DIR = "dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"
    FILES = [str(x.name) for x in dbutils.fs.ls("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv")]
    for i in range(len(FILES)):
      if(FILES[i].startswith('part')):
        dbutils.fs.cp(CSV_DIR  + 'Escalation_Log_1.csv/' + FILES[i],CSV_DIR + FILES[i])
        dbutils.fs.rm(CSV_DIR+'Escalation_Log_1.csv' , True)
        dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'Escalation_Log_1.csv')
    
    dbutils.fs.mv(CSV_DIR+'Escalation_Log_1.csv',CSV_DIR+'Escalation_Log.csv',recurse=True)
    
  else:
    counter=Log.filter((col("Report")=="Finance")&(col("Cadence")=="Periodic")).select("Count").collect()[0][0]
    if counter == 0:
      Log=Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),1).otherwise(col("Count")))
      url="https://prod-19.eastus2.logic.azure.com:443/workflows/82e53194fc454c879826eea09540296b/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=gver23_JHdc51caJeNhd1Ddo8OAcZZuUHh6rPeu_os0"
    elif counter == 1:
      Log=Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),2).otherwise(col("Count")))
      url="https://prod-57.eastus2.logic.azure.com:443/workflows/48059678ce3c4f63ae52ce013abe64fe/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=6FP04dYZj4FJWKguFpfDrqSV-xERynAZqP1q8gYMog4"
    elif counter == 2:
      Log=Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),0).otherwise(col("Count")))
      url="https://prod-38.eastus2.logic.azure.com:443/workflows/641e556114ea4636b11c302b3906a05d/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=CjIdAs-k-0RJhmMJPDhcRzkwPRxUUFd_DX97J4nOjXM"
      
  ###############################################################################################
    Log.coalesce(1).write.mode('overwrite').csv("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv",header=True)
    CSV_DIR = "dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"
    FILES = [str(x.name) for x in dbutils.fs.ls("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv")]
    for i in range(len(FILES)):
      if(FILES[i].startswith('part')):
        dbutils.fs.cp(CSV_DIR  + 'Escalation_Log_1.csv/' + FILES[i],CSV_DIR + FILES[i])
        dbutils.fs.rm(CSV_DIR+'Escalation_Log_1.csv' , True)
        dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'Escalation_Log_1.csv')
  
    dbutils.fs.mv(CSV_DIR+'Escalation_Log_1.csv',CSV_DIR+'Escalation_Log.csv',recurse=True)
    
    import json
    import urllib
    
    body = str.encode(json.dumps(1))
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url,body, headers)
    response = urllib.request.urlopen(req)
    print("QC Check fail for Brand Data File")
    FAIL
  
else:
  if (max_period==ref_period ) & (max_year==cur_year):
    Log=Log.withColumn("Flag",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),lit("1")).otherwise(col("Flag")))\
         .withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),0).otherwise(col("Count")))
    print("QC Check pass for Brand Data File")
    
    Log.coalesce(1).write.mode('overwrite').csv("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv",header=True)
    CSV_DIR = "dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"
    FILES = [str(x.name) for x in dbutils.fs.ls("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv")]
    for i in range(len(FILES)):
      if(FILES[i].startswith('part')):
        dbutils.fs.cp(CSV_DIR  + 'Escalation_Log_1.csv/' + FILES[i],CSV_DIR + FILES[i])
        dbutils.fs.rm(CSV_DIR+'Escalation_Log_1.csv' , True)
        dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'Escalation_Log_1.csv')
    
    dbutils.fs.mv(CSV_DIR+'Escalation_Log_1.csv',CSV_DIR+'Escalation_Log.csv',recurse=True)
    
    
  else:
    counter=Log.filter((col("Report")=="Finance")&(col("Cadence")=="Periodic")).select("Count").collect()[0][0]
    if counter == 0:
      Log=Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),1).otherwise(col("Count")))
      url="https://prod-19.eastus2.logic.azure.com:443/workflows/82e53194fc454c879826eea09540296b/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=gver23_JHdc51caJeNhd1Ddo8OAcZZuUHh6rPeu_os0"
    elif counter == 1:
      Log=Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),2).otherwise(col("Count")))
      url="https://prod-57.eastus2.logic.azure.com:443/workflows/48059678ce3c4f63ae52ce013abe64fe/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=6FP04dYZj4FJWKguFpfDrqSV-xERynAZqP1q8gYMog4"
    elif counter == 2:
      Log=Log.withColumn("Count",when((col("Report")=="Finance")&(col("Cadence")=="Periodic"),0).otherwise(col("Count")))
      url="https://prod-38.eastus2.logic.azure.com:443/workflows/641e556114ea4636b11c302b3906a05d/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=CjIdAs-k-0RJhmMJPDhcRzkwPRxUUFd_DX97J4nOjXM"
      
  ###############################################################################################
    Log.coalesce(1).write.mode('overwrite').csv("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv",header=True)
    CSV_DIR = "dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"
    FILES = [str(x.name) for x in dbutils.fs.ls("dbfs:/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/"+"Escalation_Log_1.csv")]
    for i in range(len(FILES)):
      if(FILES[i].startswith('part')):
        dbutils.fs.cp(CSV_DIR  + 'Escalation_Log_1.csv/' + FILES[i],CSV_DIR + FILES[i])
        dbutils.fs.rm(CSV_DIR+'Escalation_Log_1.csv' , True)
        dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'Escalation_Log_1.csv')
  
    dbutils.fs.mv(CSV_DIR+'Escalation_Log_1.csv',CSV_DIR+'Escalation_Log.csv',recurse=True)
    
    import json
    import urllib
  # Replace this with the URI and API Key for your web service
    body = str.encode(json.dumps(1))
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url,body, headers)
    response = urllib.request.urlopen(req)
    print("QC Check fail for Brand Data File")
    FAIL

Max Period Available 8

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-176462053885649> in <module> () 
 45 
 46 
 ---> 47 FAIL 

 NameError : name 'FAIL' is not defined

In [0]:

#Fix for whitespace around market names
ACTDF['Market'] = ACTDF['Market'].str.strip()
ACTDF['KPI'] = ACTDF['KPI'].str.strip()
# ACTDF.Market.unique()

In [0]:
ACTDF.dropna(subset=['Seg'], inplace=True)

In [0]:
#Melting the dataframe into the required format 
ACTDF_MELT = pd.melt(ACTDF,['Market','KPI','YEAR','PERIOD'],['Period_ACT','Period_REPLAN','YTD_ACT','YTD_REPLAN','MAT_ACT','MAT_REPLAN'])

In [0]:
#Renaming columns 
ACTDF_MELT.rename(columns = {'variable' : 'Time Range', 'value' : 'Value'}, inplace = True)


#Replace string columns
ACTDF_MELT['KPI'] = ACTDF_MELT['KPI'].str.replace('MS.','')

In [0]:
#Keep only current period
#ACTDF_MELT = ACTDF_MELT.loc[ACTDF_MELT['PERIOD']==CURR_PERIOD]

#Keep only required KPIs
LIST_OF_KPI = ['Net Sales 3rd Pty','Margin After Conversion','Advertising & Cons Promotion','Other Franch&Sales Cost','General & Admin Costs','Other Operating Costs','ROTA Earnings']

ACTDF_MELT = ACTDF_MELT.loc[ACTDF_MELT['KPI'].isin(LIST_OF_KPI)]

In [0]:
ACTDF_UNMELT = ACTDF_MELT.groupby(['Market', 'YEAR','PERIOD','Time Range','KPI']).sum().unstack('KPI')
new_col_names = ['_'.join((b,a)) for a,b in ACTDF_UNMELT.columns.get_values()]

ACTDF_UNMELT.columns = new_col_names
ACTDF_UNMELT = ACTDF_UNMELT.reset_index()
print(ACTDF_UNMELT.columns)
#Create COGS 
ACTDF_UNMELT['COGS'] = ACTDF_UNMELT['Net Sales 3rd Pty_Value'] - ACTDF_UNMELT['Margin After Conversion_Value']
ACTDF_UNMELT['MAC Negative'] = -(ACTDF_UNMELT['Margin After Conversion_Value'])
#ACTDF_UNMELT.loc[(ACTDF_UNMELT['Market']=='Mars Incorporated') & (ACTDF_UNMELT['PERIOD']==6)]

Index(['Market', 'YEAR', 'PERIOD', 'Time Range',
 'Advertising & Cons Promotion_Value', 'General & Admin Costs_Value',
 'Margin After Conversion_Value', 'Net Sales 3rd Pty_Value',
 'Other Franch&Sales Cost_Value', 'Other Operating Costs_Value',
 'ROTA Earnings_Value'],
 dtype='object')

In [0]:
# ACTDF_UNMELT.columns

In [0]:
#ACTDF_UNMELT.loc[(ACTDF_UNMELT['Market']=='Mars Incorporated') & (ACTDF_UNMELT['PERIOD']==9) , 'Net Sales 3rd Pty_Value']

In [0]:
#Change back dataframe to long format
ACTDF_MELT = pd.melt(ACTDF_UNMELT,['Market','YEAR','PERIOD','Time Range'],['Advertising & Cons Promotion_Value','General & Admin Costs_Value','Margin After Conversion_Value','Net Sales 3rd Pty_Value','Other Franch&Sales Cost_Value','Other Operating Costs_Value','ROTA Earnings_Value','COGS','MAC Negative'])

#Renaming columns 
ACTDF_MELT.rename(columns = {'variable' : 'KPI', 'value' : 'Value'}, inplace = True)


#Replace string columns
ACTDF_MELT['KPI'] = ACTDF_MELT['KPI'].str.replace('_Value','')

In [0]:
#Function to subtract and get KIND TOTAL  
def f(x):
    y = x.values
    mars_inc = y[0]
    others = sum(x for x in y[1:len(y)])
    return mars_inc - others 

In [0]:
#Fix for whitespace around market names
ACTDF_MELT['Market'] = ACTDF_MELT['Market'].str.strip()

In [0]:
ACTDF_MELT_NSV=ACTDF_MELT[ACTDF_MELT["KPI"]=="Net Sales 3rd Pty"]
ACTDF_MELT_OTH=ACTDF_MELT[ACTDF_MELT["KPI"]!="Net Sales 3rd Pty"]

In [0]:
# ACTDF_MELT['Market'].unique()

In [0]:
MARS_TOTAL_DF_NSV = ACTDF_MELT_NSV[ACTDF_MELT_NSV.Market.isin(['Mars Incorporated','KIND D&I LLC Unit','MW USA Mkt','MW USA Fct','Jiaxing Fct','Conf Hong Kong Ecommerce Mkt','KIND Corporate Office Unit'])].sort_values('Market', ascending=False)\
      .groupby(['PERIOD', 'Time Range', 'KPI', 'YEAR'], as_index=False).agg(f)\
      .assign(Market='KIND TOTAL').reset_index(drop=True)

MARS_TOTAL_DF_NSV = MARS_TOTAL_DF_NSV[['Market', 'YEAR', 'PERIOD', 'Time Range', 'KPI', 'Value']]

In [0]:
MARS_TOTAL_DF = ACTDF_MELT_OTH[ACTDF_MELT_OTH.Market.isin(['Mars Incorporated','KIND D&I LLC Unit','MW USA Mkt','MW USA Fct','Jiaxing Fct','Conf Hong Kong Ecommerce Mkt'])].sort_values('Market', ascending=False)\
      .groupby(['PERIOD', 'Time Range', 'KPI', 'YEAR'], as_index=False).agg(f)\
      .assign(Market='KIND TOTAL').reset_index(drop=True)

MARS_TOTAL_DF = MARS_TOTAL_DF[['Market', 'YEAR', 'PERIOD', 'Time Range', 'KPI', 'Value']]

In [0]:
ACTDF_MELT = pd.concat([MARS_TOTAL_DF,MARS_TOTAL_DF_NSV,ACTDF_MELT]).reset_index(drop=True)

In [0]:
ACTDF_MELT['Market'].unique()

Out[ 25 ]: array(['KIND TOTAL', 'AIME CEIPA Mkt', 'Africa', 'Andean', 'Argentina',
 'Australia', 'Austria', 'Baltics', 'Belgium', 'Brazil', 'Bulgaria',
 'CEAB', 'Canada', 'Chile', 'Chocolate USA Fct', 'Chocolate USA Mkt',
 'Conf Hong Kong Ecommerce Mkt', 'Corporate', 'Czech', 'DMEA',
 'East Med Mkt', 'East/West Balkan SCE', 'France', 'Greece', 'Gulf',
 'Hong Kong Mkt', 'Hungary', 'Iberia', 'India', 'Ireland Market',
 'Israel Mkt', 'Italy', 'Japan', 'Jiaxing Fct', 'KIND China Mkt',
 'KIND Corporate Office Unit', 'KIND D&I LLC Unit',
 'KIND Germany Mkt', 'KIND International Mkt', 'KIND Russia Mkt',
 'KIND USA Mkt', 'Levant & Maghreb Mkt', 'MCCA', 'MK Fundco Unit',
 'MK Fundco Unit-OLD', 'MW USA Fct', 'MW USA Mkt',
 'Mars Incorporated', 'Mexico', 'Mk Fundco Unit', 'Netherlands',
 'New Zealand', 'Nordics', 'Poland', 'Romania', 'Saudi Arabia',
 'Slovakia', 'South East Asia', 'Switzerland', 'Taiwan', 'Thailand',
 'Turkey', 'Uk Adj P9', 'Ukraine'], dtype=object)

In [0]:
#Map Cluster File

#Reading the cluster mapping file
mappingCLUSTER = pd.read_excel('/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/INTERNAL/KIND_INTERNAL_MAPPING.xlsx',skiprows = 2)
mappingCLUSTER = mappingCLUSTER[['KIND Regions','KIND Clusters','KIND Markets Helios','KIND Markets ']]
mappingCLUSTER.drop_duplicates(inplace=True)
mappingCLUSTER.head(3)

#Merge with mapping file
ACTDF_MELT.columns = ['KIND Markets Helios', 'YEAR', 'PERIOD', 'Time Range', 'KPI', 'Value']
OUTPUT_DF = pd.merge(ACTDF_MELT,mappingCLUSTER,how='left')

In [0]:
# OUTPUT_DF['KIND Markets Helios'].unique()

Out[ 29 ]: array(['KIND TOTAL', 'AIME CEIPA Mkt', 'Africa', 'Andean', 'Argentina',
 'Australia', 'Austria', 'Baltics', 'Belgium', 'Brazil', 'Bulgaria',
 'CEAB', 'Canada', 'Chile', 'Chocolate USA Fct', 'Chocolate USA Mkt',
 'Conf Hong Kong Ecommerce Mkt', 'Corporate', 'Czech', 'DMEA',
 'East Med Mkt', 'East/West Balkan SCE', 'France', 'Greece', 'Gulf',
 'Hong Kong Mkt', 'Hungary', 'Iberia', 'India', 'Ireland Market',
 'Israel Mkt', 'Italy', 'Japan', 'Jiaxing Fct', 'KIND China Mkt',
 'KIND Corporate Office Unit', 'KIND D&I LLC Unit',
 'KIND Germany Mkt', 'KIND International Mkt', 'KIND Russia Mkt',
 'KIND USA Mkt', 'Levant & Maghreb Mkt', 'MCCA', 'MK Fundco Unit',
 'MK Fundco Unit-OLD', 'MW USA Fct', 'MW USA Mkt',
 'Mars Incorporated', 'Mexico', 'Mk Fundco Unit', 'Netherlands',
 'New Zealand', 'Nordics', 'Poland', 'Romania', 'Saudi Arabia',
 'Slovakia', 'South East Asia', 'Switzerland', 'Taiwan', 'Thailand',
 'Turkey', 'Uk Adj P9', 'Ukraine'], dtype=object)

In [0]:
list(set(list(OUTPUT_DF.loc[OUTPUT_DF["KIND Regions"].isnull(), 'KIND Markets Helios'])))

Out[ 30 ]: ['Mk Fundco Unit',
 'Jiaxing Fct',
 'Corporate',
 'Chocolate USA Mkt',
 'KIND TOTAL',
 'Mars Incorporated',
 'MW USA Fct',
 'Gulf',
 'MK Fundco Unit-OLD',
 'Uk Adj P9',
 'MW USA Mkt',
 'DMEA']

In [0]:
OUTPUT_DF = OUTPUT_DF.loc[~OUTPUT_DF['KIND Markets Helios'].isin(['MW USA Mkt','Jiaxing Fct','Corporate','Mars Incorporated'])]

In [0]:
list(set(list(OUTPUT_DF.loc[OUTPUT_DF["KIND Regions"].isnull(), 'KIND Markets Helios'])))

Out[ 32 ]: ['Mk Fundco Unit',
 'Chocolate USA Mkt',
 'KIND TOTAL',
 'MW USA Fct',
 'Gulf',
 'MK Fundco Unit-OLD',
 'Uk Adj P9',
 'DMEA']

In [0]:
#Custom Cluster, region and market names
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios'] == 'KIND TOTAL', 'KIND Regions'] = 'KIND TOTAL'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios'] == 'KIND TOTAL', 'KIND Clusters'] = 'KIND TOTAL'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios'] == 'KIND TOTAL', 'KIND Markets '] = 'KIND TOTAL'


OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Uk Adj P9', 'KIND Clusters']= 'UK/Ireland'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Uk Adj P9', 'KIND Regions']= 'UK/Ireland'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Uk Adj P9', 'KIND Markets ']= 'UK'

OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='DMEA', 'KIND Clusters']= 'ROW TSA'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='DMEA', 'KIND Regions']= 'ROW'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='DMEA', 'KIND Markets ']= 'TSA'

OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Gulf', 'KIND Clusters']= 'GCC'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Gulf', 'KIND Regions']= 'ROW'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Gulf', 'KIND Markets ']= 'Gulf/UAE'

OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MK Fundco Unit-OLD', 'KIND Clusters']= 'ROW TSA'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MK Fundco Unit-OLD', 'KIND Regions']= 'ROW'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MK Fundco Unit-OLD', 'KIND Markets ']= 'TSA'

OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MK Fundco Unit', 'KIND Clusters']= 'ROW TSA'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MK Fundco Unit', 'KIND Regions']= 'ROW'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MK Fundco Unit', 'KIND Markets ']= 'TSA'

OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='MW USA Fct', 'KIND Markets ']= 'MW USA Fct'
OUTPUT_DF.loc[OUTPUT_DF['KIND Markets Helios']=='Chocolate USA Mkt', 'KIND Markets ']= 'Chocolate USA Mkt'

In [0]:
#CHange Margin After Conversion to MAC
OUTPUT_DF.loc[OUTPUT_DF['KPI']== 'Margin After Conversion', 'KPI'] = 'MAC'

In [0]:
#Round to 2 digits
# OUTPUT_DF['Value'] = OUTPUT_DF['Value'].round(2)

In [0]:
# OUTPUT_DF['KIND Markets Helios'].unique()

In [0]:
#Defining the schema to convert the Pandas dataframe into a spark dataframe
mySchema = StructType([StructField("KIND Markets Helios", StringType(), True)\
                       ,StructField("YEAR", IntegerType(), True)\
                       ,StructField("PERIOD",IntegerType(),True)\
                       ,StructField("Time Range", StringType(), True)\
                       ,StructField("KPI", StringType(), True)\
                       ,StructField("Value", DoubleType(), True)\
                       ,StructField("KIND Regions", StringType(), True)\
                       ,StructField("KIND Clusters", StringType(), True)\
                       ,StructField("KIND Markets", StringType(), True)])
 
#Creating a spark dataframe
ACTUALDF = spark.createDataFrame(OUTPUT_DF,schema = mySchema)

In [0]:
# display(ACTUALDF.filter(col("KIND Markets Helios")=="Slovakia"))

In [0]:
#Replace 0 with lit('None')
def replace(column, value):
    return when(column != value, column).otherwise(lit(''))

ACTUALDF = ACTUALDF.withColumn("Value", replace(col("Value"), 'NaN'))

In [0]:
#treating NAN values
columns=ACTUALDF.columns
for column in columns:
    ACTUALDF = ACTUALDF.withColumn(column,F.when(F.isnan(F.col(column)),0).otherwise(F.col(column)))

In [0]:
# display(ACTUALDF.filter(col("KIND Markets Helios")=="Slovakia"))

In [0]:
# Write Final dataframe
ACTUALDF.coalesce(1).write.mode('overwrite').csv('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/WATERFALL_TABLE.csv',header=True)
CSV_DIR = 'dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/'
FILES = [str(x.name) for x in dbutils.fs.ls('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/WATERFALL_TABLE.csv')]
for i in range(len(FILES)):
  if(FILES[i].startswith('part')):
          dbutils.fs.cp(CSV_DIR  + 'WATERFALL_TABLE.csv/' + FILES[i],CSV_DIR + FILES[i])
          dbutils.fs.rm(CSV_DIR+'WATERFALL_TABLE.csv' , True)
          dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'WATERFALL_TABLE.csv')

In [0]:
OUTPUT_DF.loc[OUTPUT_DF['KPI']=='Other Franch&Sales Cost' , 'Value'] = OUTPUT_DF['Value'] * -1
OUTPUT_DF.loc[OUTPUT_DF['KPI']=='Other Operating Costs' , 'Value'] = OUTPUT_DF['Value'] * -1
OUTPUT_DF.loc[OUTPUT_DF['KPI']=='Advertising & Cons Promotion' , 'Value'] = OUTPUT_DF['Value'] * -1
OUTPUT_DF.loc[OUTPUT_DF['KPI']=='General & Admin Costs' , 'Value'] = OUTPUT_DF['Value'] * -1
OUTPUT_DF.loc[OUTPUT_DF['KPI']=='COGS' , 'Value'] = OUTPUT_DF['Value'] * -1

In [0]:
#Creating a spark dataframe
ACTUALDF_NEG_COSTS = spark.createDataFrame(OUTPUT_DF,schema = mySchema)

#treating NAN values
columns=ACTUALDF_NEG_COSTS.columns
for column in columns:
    ACTUALDF_NEG_COSTS = ACTUALDF_NEG_COSTS.withColumn(column,F.when(F.isnan(F.col(column)),0).otherwise(F.col(column)))

# Write Final dataframe
ACTUALDF_NEG_COSTS.coalesce(1).write.mode('overwrite').csv('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/WATERFALL_TABLE_TABLEAU.csv',header=True)
CSV_DIR = 'dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/'
FILES = [str(x.name) for x in dbutils.fs.ls('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/WATERFALL_TABLE_TABLEAU.csv')]
for i in range(len(FILES)):
  if(FILES[i].startswith('part')):
          dbutils.fs.cp(CSV_DIR  + 'WATERFALL_TABLE_TABLEAU.csv/' + FILES[i],CSV_DIR + FILES[i])
          dbutils.fs.rm(CSV_DIR+'WATERFALL_TABLE_TABLEAU.csv' , True)
          dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'WATERFALL_TABLE_TABLEAU.csv')

In [0]:
display(ACTUALDF_NEG_COSTS)

KIND Markets Helios,YEAR,PERIOD,Time Range,KPI,Value,KIND Regions,KIND Clusters,KIND Markets
KIND TOTAL,2019,1,MAT_ACT,Advertising & Cons Promotion,-0.5514593033602195,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2020,1,MAT_ACT,Advertising & Cons Promotion,-30.54172341154965,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2019,1,MAT_ACT,COGS,-8.83754268199572,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2020,1,MAT_ACT,COGS,-34.70983309160768,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2019,1,MAT_ACT,General & Admin Costs,-19.11482884969233,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2020,1,MAT_ACT,General & Admin Costs,-36.50578088655304,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2019,1,MAT_ACT,MAC Negative,-3.380208395072123,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2020,1,MAT_ACT,MAC Negative,-14.155312615805569,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2019,1,MAT_ACT,MAC,3.380208395072123,KIND TOTAL,KIND TOTAL,KIND TOTAL
KIND TOTAL,2020,1,MAT_ACT,MAC,14.155312615805569,KIND TOTAL,KIND TOTAL,KIND TOTAL
